In [1]:

import numpy as np
import math
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister,Aer, transpile, execute
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import MCMT
from qiskit.circuit.library import QFT
import matplotlib.pyplot as plt

In [2]:
def create_lists(clause):
    pos_var = []
    neg_var = []
    none = []
    for index,item in enumerate(clause):
        if item>0:
            pos_var.append(index)
        elif item<0:
            neg_var.append(index)
        else:
             none.append(index)
    return pos_var,neg_var,none

In [3]:
def encode_clause(clause,circuit,qreg):
    
    num_of_variables = len(clause)
    positive_var,negative_var,either_var = create_lists(clause)
    
    #Building the oracle
    
    #negating the variables
    if len(negative_var)!=0:
        circuit.x(qreg[i] for i in negative_var)
        circuit.barrier()

    #Logical OR operation
    circuit.x([qreg[i] for i in (positive_var+negative_var)])
    #qc.mct([qreg[i] for i in range(num_of_variables)],qr[[-1]])
    circuit.mct([qreg[i] for i in (positive_var+negative_var)],qreg[-1])
    circuit.x([qreg[i] for i in (positive_var+negative_var)])
    circuit.barrier()

    #Bringing the negated variables to original form
    if len(negative_var)!=0:
        circuit.x(qreg[i] for i in negative_var)
        circuit.barrier()
    
    return circuit

In [4]:
def diffusion_circ(list):
    num_of_variables = len(list)
    
    qr = QuantumRegister(num_of_variables+1)
    cr = ClassicalRegister(num_of_variables+1)
    qc = QuantumCircuit(qr,cr)
    
    qc.h(qr[:])
    qc.x(qr[:])
    qc.h(qr[-1])
    qc.mct([qr[i] for i in range(num_of_variables)],qr[[-1]])
    qc.h(qr[-1])
    qc.x(qr[:])
    qc.h(qr[:])
    
    return qc

### 3-SAT problem

In [5]:
def hybrid_3mct(circuit,clause,clause_number,aux,aux1,f_in,f_out):
    
    control_qbits = [abs(val) for i,val in enumerate(clause) if val!=0]
    circuit.ccx(f_in[control_qbits[0]-1],f_in[control_qbits[1]-1],aux1[0])
    circuit.ccx(f_in[control_qbits[2]-1],aux1[0],aux[clause_number])
    circuit.ccx(f_in[control_qbits[0]-1],f_in[control_qbits[1]-1],aux1[0])
    
    return circuit


def hybrid_oracle(circuit,f_in,f_out,aux,aux1,n,sat_formula):
    
    for j in range(len(sat_formula)):
        clause = sat_formula[j]    
        for i in range(len(clause)):
            #Encoding positive variables
            if clause[i]>0:
                circuit.x(f_in[clause[i]-1])
        hybrid_3mct(circuit,clause,j,aux,aux1,f_in,f_out)
        circuit.x(aux[j])
        for i in range(len(clause)):
            #Encoding positive variables
            if clause[i]>0:
                circuit.x(f_in[clause[i]-1])
        #circuit.barrier()
        
    return circuit  


def diffusion_circ3(circuit,f_in,aux2):
    circuit.h(f_in[:])
    circuit.x(f_in[:])
    circuit.h(f_in[-1])
    if len(f_in[:])==3:
        circuit.ccx(f_in[0],f_in[1],f_in[-1])
    elif len(f_in[:])==4:
        circuit.ccx(f_in[0],f_in[1],aux2[0])
        circuit.ccx(aux2[0],f_in[2],f_in[-1])
        circuit.ccx(f_in[0],f_in[1],aux2[0])
    elif len(f_in[:])==5:
        circuit.ccx(f_in[0],f_in[1],aux2[0])
        circuit.ccx(f_in[2],f_in[3],aux2[1])
        circuit.ccx(aux2[0],aux2[1],f_in[-1])
        circuit.ccx(f_in[0],f_in[1],aux2[0])
        circuit.ccx(f_in[2],f_in[3],aux2[1])
    circuit.h(f_in[-1])
    circuit.x(f_in[:])
    circuit.h(f_in[:])
    #circuit.barrier()
    
    return circuit

In [6]:
def hybrid_sat3(sat_formula):
    n = max(max(sat_clause) for sat_clause in sat_formula)
    f_in = QuantumRegister(n)
    f_out = QuantumRegister(1)
    aux = QuantumRegister(len(sat_formula))
    aux1 = QuantumRegister(1)
    aux2 = QuantumRegister(2)
    cr = ClassicalRegister(n)
    circuit = QuantumCircuit(f_in,f_out,aux,aux1,aux2,cr)
    print(n,len(sat_formula))
    #Superposition of input states
    for i in range(n):
        circuit.h(f_in[i])
    # circuit.x(aux[1])
    # circuit.h(aux[1])
    #initializing f_out qubit with state |1>
    circuit.x(f_out[0])
    circuit.barrier()

    print(math.ceil(np.pi*math.sqrt(2**n/5)/4 - 0.5)-1)
    #oracle
    if len(sat_formula)==3:
        for _ in range(math.ceil(np.pi*math.sqrt(2**n/12)/4 - 0.5)):
            hybrid_oracle(circuit,f_in,f_out,aux,aux1,n,sat_formula)
            circuit.ccx(aux[0],aux[1],aux1[0])
            circuit.ccz(aux[2],aux1[0],f_out[0])
            circuit.ccx(aux[0],aux[1],aux1[0])
            
            circuit.barrier()
            diffusion_circ3(circuit,f_in,aux2)
        circuit.measure(f_in[:],cr[:])
            
        return circuit
    else:
        cr=ClassicalRegister(n)
        sat_circuit = QuantumCircuit(f_in,f_out,aux,aux1,aux2,cr)
        hybrid_oracle(circuit,f_in,f_out,aux,aux1,n,sat_formula)
        cnz_circuit = MCMT('z',len(sat_formula),1)
        sat_circuit.compose(circuit,list(range(0,n+1+len(sat_formula)+1+2)),list(range(0,n)),inplace=True)
        print(cnz_circuit.num_qubits)
        print(n+1+len(sat_formula)+1+cnz_circuit.num_qubits)
        sat_circuit.compose(cnz_circuit,list(range(n,n+cnz_circuit.num_qubits)),inplace=True)
        sat_circuit.barrier()
        diffusion_circ3(sat_circuit,f_in,aux2)
        
        for _ in range(math.ceil(np.pi*math.sqrt(2**n/12)/4 - 0.5)):
            sat_circuit = hybrid_oracle(sat_circuit,f_in,f_out,aux,aux1,n,sat_formula)
            cnz_circuit = MCMT('z',len(sat_formula),1)
            sat_circuit.compose(cnz_circuit,list(range(n,n+cnz_circuit.num_qubits)),inplace=True)
            sat_circuit.barrier()
            diffusion_circ3(sat_circuit,f_in,aux2)
        sat_circuit.measure(f_in[:],cr[:])

    
        return sat_circuit
        

In [7]:
#Generating random k-SAT problem
import random
def random_kcnf(n_literals, n_conjuncts, k=3):
    result = []
    for _ in range(n_conjuncts):
        sat_clause = random.sample(range(1,n_literals+1),3)
        k = random.randint(0,4)
        if k==1:
            pick_var = random.randint(0,3)
            sat_clause[pick_var] = -sat_clause[pick_var]
        elif k==2:
            pick_var = random.sample(range(0,3),2)
            sat_clause[pick_var[0]] = -sat_clause[pick_var[0]]
            sat_clause[pick_var[1]] = -sat_clause[pick_var[1]]
        elif k==3:
            sat_clause = [-x for x in sat_clause]
        elif k==0:
            sat_clause = sat_clause
        #print(sat_clause)
        sat_clause = sorted(sat_clause,key=abs) 
        result.append(sat_clause)
    return result

In [8]:
def classical_sat_checker(sat_formula,quantum_solution):
    
    sat_solution = []
    for k in range(len(quantum_solution)):              #for each clause in a classical part of sat_formula
        final_result = []
        for sat_clause in sat_formula:                  #for each solution from set of quantum valid solutions
            clause_result = []
            for variable in sat_clause:   #For a each sat clause and each quantum solution we zip each literals correspondingly
                quant_variable = quantum_solution[k][np.abs(variable)-1]
                #print(variable,quant_variable)
                if variable>0 and int(quant_variable)>0:
                    clause_result.append('yes')
                elif variable<0 and int(quant_variable)==0:
                    clause_result.append('yes')
                else:
                    clause_result.append('no')
            # print(clause_result)
            if 'yes' in clause_result:
                final_result.append('yes')
            else:
                final_result.append('no')
        all_yes = all(element == 'yes' for element in final_result)
        #print(final_result)
        if all_yes:
            sat_solution.append(quantum_solution[k])                     #Assuming we have only one sat solution

    if len(sat_solution)==0:
        sat_solution.append('No solution exists')
    return(sat_solution)


#### Quantum Counting Algorithm

In [9]:
def cg_operator(sat_formula,m,e):
    
    #Register 2
    n = max(max(sat_clause) for sat_clause in sat_formula)
    f_in = QuantumRegister(n)
    f_out = QuantumRegister(1)
    aux = QuantumRegister(len(sat_formula))
    aux1 = QuantumRegister(1)
    aux2 = QuantumRegister(2)
    reg2 = n+1+len(sat_formula)+1+2
    
    #Empty circuit
    circuit = QuantumCircuit(f_in,f_out,aux,aux1,aux2)
    
    #Applying oracle on the circuit
    circuit = hybrid_oracle(circuit,f_in,f_out,aux,aux1,n,sat_formula)
    
    #Applying diffusion gate on the circuit
    circuit = diffusion_circ3(circuit,f_in,aux2)
    
    #Changing the circuit to control gate
    cg_gate = circuit.to_gate().control()
    cg_gate.label = 'C_G'
    
    return cg_gate

In [10]:
def quantum_counting(sat_formula, m, e):   #m = requiered accuracy to determine M
                                           #e = required probability (1-e) to obtain M
    
    #Register 1
    reg1 = m + math.floor(math.log(2+0.5/e))
    t = QuantumRegister(reg1)
    
    #Register 2
    n = max(max(sat_clause) for sat_clause in sat_formula)
    f_in = QuantumRegister(n)
    f_out = QuantumRegister(1)
    aux = QuantumRegister(len(sat_formula))
    aux1 = QuantumRegister(1)
    aux2 = QuantumRegister(2)
    cr = ClassicalRegister(reg1)
    
    reg2 = n+1+len(sat_formula)+1+2
    
    circuit = QuantumCircuit(t,f_in,f_out,aux,aux1,aux2,cr)
    
    #Initial Superposition and set up
    circuit.h([t[i] for i in range(reg1)])
    circuit.h([f_in[i] for i in range(n)])
    circuit.x(f_out[0])
    circuit.barrier()
    
    #Adding control gates
    for i in range(reg1):
        exponent = 2**(reg1-i-1)
        index_list = [i]
        index_list += list(range(reg1,reg1+reg2))
        
        for j in range(exponent):
            circuit.append(cg_operator(sat_formula,m,e),index_list)
        
        circuit.barrier()
        
    #     index_list = [i]
    #     index_list += list(range(reg1,reg1+reg2))
    #     circuit.append(c_grover(sat_formula,i),index_list)
    # circuit.barrier()
    
    #Inverse QFT
    circuit.append(QFT(reg1).inverse(),list(range(0,reg1)))
    circuit.barrier()
    
    #Measurement
    circuit.measure(t[:],cr[:])
    
    return circuit